In [2]:
import zipfile
import os
import xml.etree.ElementTree as ET
import csv
import time
from urllib.request import urlretrieve
from datetime import date
from calendar import monthrange

import pandas as pd
import requests

In [3]:
#recupération des bases de donnée sur le site du gouvernement.
def recuperation_xml(date_debut,date_fin):
    for date in range(date_debut, date_fin +1, 1):
        directory_to_extract_to = os.path.join("unzip_file")
        path_to_zip_file  = os.path.join("zip_file",f"PrixCarburants_annuel_{date}.zip")
        urlretrieve(f"https://donnees.roulez-eco.fr/opendata/annee/{date}", path_to_zip_file)
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract_to)
#recuperation_xml(2007,2021)

In [4]:
# utilisation de l'API de adress.data.gouv.fr pour passer de la latitude et longitude, au citycode
def citycode_from_lat_long(longitude,latitude):
    url = f"https://api-adresse.data.gouv.fr/reverse/?lon={longitude}&lat={latitude}"
    response = requests.get(url)
    contenu = response.json() 
    features = contenu['features']
    if len(features) == 0:
        return None
    else:
        citycode = contenu['features'][0]['properties']['citycode']
        return citycode

In [5]:
# passage du citycode au code du departement
def code_departement_from_citycode(citycode):
    if citycode[ : 2] >= '97':
        code_departement = citycode[ : 3]
    else:
        code_departement = citycode[ : 2]
    return code_departement

In [6]:
# passage du code postal au code du departement
def code_departement_from_code_postal(code_postal):
    if code_postal == '99999':
        return None
    elif code_postal[ : 2] >= '97':
        code_departement = code_postal[ : 3]
    elif code_postal[ : 3] in ["200","201"] :
        code_departement = "2A"
    elif code_postal[ : 3] in ["202","206"]:
        code_departement = "2B"
    else:
        code_departement = code_postal[ : 2]     
    return code_departement

In [7]:
# passage du code du departement au code région en utilisant l'API Métadonnées - V1 de l'INSEE
# documentation à API nomenclatures géographiques Insee
# attention, la clé doit être réactualisé tous les 7 jours...
# l'API est limité à 30 requêtes par minute
def code_region_from_code_departement(code_departement,date):
    headers = {
        'Accept': 'application/json',
        'Authorization': 'Bearer ################', #Le changement est ici
    }
    params = {
        'date': date,
    }
    response = requests.get(f'https://api.insee.fr/metadonnees/V1/geo/departement/{code_departement}/ascendants', params=params, headers=headers)
    contenu = response.json()
    time.sleep(2.1)
    if isinstance(contenu,dict):
        print(contenu)
    return contenu[0]['code']

In [8]:
#Les APIs sont relativement fragile, il arrive qu'il y ai des erreurs 500 ou 502. 
#Dans le cas là if faut supprimer l'année qui était en train de boucler de "prix_by_region".
#Il faut ensuite recommencer la boucle à partir de cette date.
def debug_if_error_500(date_debut,date_fin):
    for region, prix_by_carburant in prix_by_region.items():
        for carburant,prix_by_annee in prix_by_carburant.items():
            for annee in range(date_debut,date_fin+1):
                if annee in prix_by_annee:
                    del prix_by_annee[annee]
#debug_if_error_500(2007,2007)

In [9]:
tree = ET.parse('unzip_file/PrixCarburants_annuel_2021.xml')
pdv_liste = tree.getroot()

In [10]:
citycode_lat_long = {} 

In [11]:
prix_by_region = {}

In [52]:
#boucle principale, qui récupére les données des fichiers XML,
#trouve le code région de chaque station, 
#récupère les données importantes, dont le prix par jour, par carburant, par station,
#nous avons uniquement les prix des jours ou il y a eu un changement, il faut créer un prix aux jours ou il n'y en a pas eu,
#fait la moyenne par jour de toutes les stations,
#fait la moyenne par région, par mois et par annee, des prix des différents carburants.
for annee in range(2007,2022):
    print(annee)
    tree = ET.parse(f'unzip_file/PrixCarburants_annuel_{annee}.xml')
    pdv_liste = tree.getroot()
    date = f'{annee}-01-01'
    region = {}    
    for pdv in pdv_liste:
        longitude = pdv.attrib.get('longitude')
        latitude = pdv.attrib.get('latitude')
        citycode = None
        if latitude and longitude:
            lat_long = f"{latitude},{longitude}"       
        if latitude and longitude:
            lat_long = f"{latitude},{longitude}"
            citycode = citycode_lat_long.get(lat_long)
            if citycode is None:
                citycode = citycode_from_lat_long(float(longitude)/100000,float(latitude)/100000)
                if citycode is not None:
                    citycode_lat_long[lat_long] = citycode                 
        code_departement = (
            code_departement_from_code_postal(pdv.attrib['cp'])
            if citycode is None
            else code_departement_from_citycode(citycode)
        )
        if code_departement is None:
            print('code_departement is None')
            continue
        code_region = region.get(code_departement) 
        if code_region is None:
            code_region = code_region_from_code_departement(code_departement,date)
            region[code_departement]= code_region
        for prix_element in pdv:
            if prix_element.tag != 'prix':
                continue
            if prix_element.attrib.get('maj') is None:
                continue
            if prix_element.attrib.get('nom') is None:
                continue
            if prix_element.attrib.get('valeur') is None:
                continue
            prix_by_carburant = prix_by_region.setdefault(code_region,{})
#            prix_by_carburant = prix_by_region.get(code_region)
#            if prix_by_carburant is None:
#                prix_by_carburant = prix_by_region[code_region] = {}
            if 'T' in prix_element.attrib['maj']:
                date_prix = prix_element.attrib['maj'].split('T')[0]
            else:
                date_prix = prix_element.attrib['maj'].split(' ')[0]
            annee_prix, mois_prix, jour_prix = date_prix.split('-')
            annee_prix, mois_prix, jour_prix = int(annee_prix), int(mois_prix), int(jour_prix)
            prix_by_annee = prix_by_carburant.setdefault(prix_element.attrib['nom'],{})
            prix_by_mois = prix_by_annee.setdefault(annee_prix,{})
            prix_by_jour = prix_by_mois.setdefault(mois_prix,{})
            prix_by_station = prix_by_jour.setdefault(jour_prix,{})
            prix_by_station[pdv.attrib['id']] = prix_element.attrib['valeur']
            
    for region, prix_by_carburant in prix_by_region.items():
        stations = set()
        prix_by_carburant = prix_by_region[region] 
        for carburant,prix_by_annee in prix_by_carburant.items():
            dernier_prix_par_station = {}
            prix_by_mois = prix_by_annee.setdefault(annee,{})
            for mois in range(1,13):
                prix_by_jour = prix_by_mois.setdefault(mois,{})
                dernier_jour = monthrange(annee, mois)[1]
                for jour in range(1,dernier_jour+1):
                    prix_by_station = prix_by_jour.setdefault(jour,{})
                    stations = stations.union(prix_by_station.keys())
                    for station in stations:
                        prix = prix_by_station.get(station)
                        if prix is None:
                            prix_by_station[station] = dernier_prix_par_station.get(station)
                        else:
                            dernier_prix_par_station[station] = prix

    for region, prix_by_carburant in prix_by_region.items():
        for carburant, prix_by_annee in prix_by_carburant.items():
            prix_by_mois = prix_by_annee.setdefault(annee,{})
            for mois, prix_by_jour in prix_by_mois.items(): 
                for jour, prix_by_station in prix_by_jour.items():
                    count = 0
                    total = 0
                    for station, prix in prix_by_station.items():
                        if prix is not None:
                            total += float(prix)
                            count += 1 
                    prix_by_jour[jour] = round(total / count, 2) if count > 0 else None

    for region, prix_by_carburant in prix_by_region.items():
        for carburant,prix_by_annee in prix_by_carburant.items():
            prix_by_mois = prix_by_annee[annee]
            count_annee = 0
            total_annee = 0
            for mois,prix_by_jour in prix_by_mois.items():
                count_mois = 0
                total_mois = 0
                for jour, prix in prix_by_jour.items():
                    if prix is not None:
                        count_mois += 1
                        total_mois += prix
                        count_annee += 1
                        total_annee += prix
                if count_mois == 0:
                    prix_by_mois[mois] = None
                else:
                    prix_by_mois[mois] = round(total_mois / count_mois,2)
            if count_annee == 0:
                prix_by_mois['moyenne'] = None
            else:
                prix_by_mois['moyenne'] = round(total_annee / count_annee,2)

2020
2021


In [53]:
#Lisse le dictionnaire "prix_by_region".
liste_prix_mensuel=[] 
liste_prix_annuel=[]
for region, prix_by_carburant in prix_by_region.items():
    for carburant,prix_by_annee in prix_by_carburant.items():
        for annee,prix_by_mois in prix_by_annee.items():
            for mois,prix in prix_by_mois.items():
                if prix_by_mois.values == 'moyenne':
                    pass
                prix_region_mensuel = {
                            "region": region,
                            "carburant": carburant,
                            "annee": annee,
                            "mois": mois,
                            "prix_moyen": prix,
                        }
                liste_prix_mensuel.append(prix_region_mensuel)

In [71]:
#créer la dataframe "prix_mensuel_carburants_par_regions_litre.csv"
df = pd.DataFrame.from_dict(liste_prix_mensuel)
indexNames = df[ df['mois'] == 'moyenne' ].index
df.drop(indexNames , inplace=True)
df.reset_index(drop = True, inplace = True)
df['prix_moyen'] = round(df['prix_moyen'] * 0.001,2)
df.rename(columns = {'prix_moyen':'prix_moyen_by_litre'}, inplace = True)
df.to_csv(r'prix_mensuel_carburants_par_regions_litre.csv', index = False, header=True)

In [72]:
#créer la dataframe "prix_annuel_carburants_par_regions_litre.csv"
df = pd.DataFrame.from_dict(liste_prix_mensuel)
indexNames = df[ df['mois'] != 'moyenne' ].index
df.drop(indexNames , inplace=True)
df.reset_index(drop = True, inplace = True)
df.drop(columns=['mois'],inplace=True)
df['prix_moyen'] = round(df['prix_moyen'] * 0.001,2)
df.rename(columns = {'prix_moyen':'prix_moyen_par_litre'}, inplace = True)
df.to_csv(r'prix_annuel_carburants_par_regions_litre.csv', index = False, header=True)

In [73]:
#créer la dataframe "prix_mensuel_carburants_par_regions_hectolitre.csv"
df = pd.DataFrame.from_dict(liste_prix_mensuel)
indexNames = df[ df['mois'] == 'moyenne' ].index
df.drop(indexNames , inplace=True)
df.reset_index(drop = True, inplace = True)
df['prix_moyen'] = round(df['prix_moyen'] * 0.1,2)
df.rename(columns = {'prix_moyen':'prix_moyen_par_hectolitre'}, inplace = True)
df.to_csv(r'prix_mensuel_carburants_par_regions_hectolitre.csv', index = False, header=True)

In [74]:
#créer la dataframe "prix_annuel_carburants_par_regions_hectolitre.csv"
df = pd.DataFrame.from_dict(liste_prix_mensuel)
indexNames = df[ df['mois'] != 'moyenne' ].index
df.drop(indexNames , inplace=True)
df.reset_index(drop = True, inplace = True)
df.drop(columns=['mois'],inplace=True)
df['prix_moyen'] = round(df['prix_moyen'] * 0.1,2)
df.rename(columns = {'prix_moyen':'prix_moyen_par_hectolitre'}, inplace = True)
df.to_csv(r'prix_annuel_carburants_par_regions_hectolitre.csv', index = False, header=True)

In [52]:
#agrege les prix au niveau national, pour pouvoir les verifier par rapport aux données de l'INSEE, et voir si il y a une coeherence.
df_ann = pd.read_csv("prix_annuel_carburants_par_regions_litre.csv", sep=",")
df_ann = df_ann.groupby(['carburant','annee'])[['prix_moyen_par_litre']].mean().reset_index().round(3)
df_ann.to_csv(r'prix_par_carburant_annee.csv',index = False, header=True)

In [18]:
#agrege les prix au niveau national, pour pouvoir les verifier par rapport aux données de l'INSEE, et voir si il y a une coeherence.
df_mens = pd.read_csv("prix_mensuel_carburants_par_regions_litre.csv", sep=",")
df_mens = df_mens.groupby(['carburant','annee','mois'])[['prix_moyen_by_litre']].mean().reset_index().round(3)
df_mens.to_csv(r'prix_par_carburant_mois.csv',index = False, header=True)